# Read the input Data

In [1]:
import pandas as pd
import numpy as np
import math
import copy as cp
import sys
import pickle
train_raw = pd.read_csv("./concrete/train.csv", header=None).values
cols = train_raw.shape[1]
rows = train_raw.shape[0]
train_x = np.copy(train_raw)
train_x[:,cols - 1] = 1
train_y = train_raw[:, cols - 1]

test_raw = pd.read_csv("./concrete/test.csv", header=None).values
cols = test_raw.shape[1]
rows = test_raw.shape[0]
test_x = np.copy(test_raw)
test_x[:,cols - 1] = 1
test_y = test_raw[:, cols - 1]

In [2]:
print(np.reshape(train_y, (-1,1)).shape)

(53, 1)


# Batched LMS Implementation

In [3]:
def Gradien_Decent(x,y):
    m = x.shape[0]
    n = x.shape[1]
    Error = 1
    threshold = 10e-6
    Max_iter = 10000
    LearningRate = 0.01
    w = np.zeros([n,1])
    Costs = []
    iter = 0
    while Error > threshold and iter < Max_iter:
        iter = iter + 1
        diff = np.matmul(x, w) - np.reshape(y, (-1,1))      
        gradient = np.sum(x * (diff), axis = 0)
        w1 = w - LearningRate*np.reshape(gradient,(-1,1))
        Error = np.sum(np.abs(w - w1))
        w = w1
        cost = 0.5 * np.sum(np.square(diff))
        Costs.append([iter, cost])
    print("converged after ", iter, "iterations.")
    return w, Costs

In [4]:
w, Costs = Gradien_Decent(train_x, train_y)
print("Batched Gradien_Decent W vector:")
print(w)
from numpy import savetxt
savetxt('Costs.csv', Costs, delimiter=',')


converged after  5859 iterations.
Batched Gradien_Decent W vector:
[[ 0.89919933]
 [ 0.78488638]
 [ 0.84952298]
 [ 1.29780387]
 [ 0.12966251]
 [ 1.57041505]
 [ 0.99730037]
 [-0.01522461]]


In [5]:
Test_Cost = 0.5 * np.sum(np.square(np.reshape(np.squeeze(np.matmul(test_x,w)) - test_y, (-1,1))))
print('Batched Test_Cost:', Test_Cost)

Batched Test_Cost: 23.359919165485493


# Stocastic LMS Implementation

In [6]:
def Stoc_Gradien_Decent(x,y):
    m = x.shape[0]
    n = x.shape[1]
    Error = 1
    threshold = 10e-6
    Max_iter = 100000
    LearningRate = 0.01
    w = np.zeros([n,1])
    Costs = []
    iter = 0
    while Error > threshold and iter < Max_iter:
        iter = iter + 1
        idx = np.random.randint(m,size=1)
        x1 = x[idx]
        y1 = y[idx]
        SG = (y1 - np.dot(x1, w))*x1 
        w1 = w + LearningRate* np.reshape(SG, (-1,1))
        Error = np.sum(np.abs(w - w1))
        w = w1
        cost = 0.5 * np.sum(np.square(np.matmul(x, w) - np.reshape(y, (-1,1))))
        Costs.append([iter, cost])
    print("converged after ", iter, "iterations.")
    return w, Costs

In [7]:
w, Costs = Stoc_Gradien_Decent(train_x, train_y)
print("Stocastic Gradien_Decent W vector:")
print(w)
from numpy import savetxt
savetxt('Stoc_Costs.csv', Costs, delimiter=',')

converged after  31982 iterations.
Stocastic Gradien_Decent W vector:
[[ 0.40661914]
 [ 0.28404155]
 [ 0.27415111]
 [ 0.86267859]
 [-0.02599699]
 [ 0.85171128]
 [ 0.4460168 ]
 [ 0.14977482]]


In [9]:
Test_Cost = 0.5 * np.sum(np.square(np.reshape(np.squeeze(np.matmul(test_x,w)) - test_y, (-1,1))))
print('Stocastic Test_Cost:', Test_Cost)

Stocastic Test_Cost: 23.180313947184068


# Analytical LMS Implementation

In [12]:
def Analytical_Gradien_Decent(x, y):
    x_t = np.transpose(x)
    return np.matmul(np.linalg.inv(np.matmul(x_t, x)),np.matmul(x_t, y))

In [14]:
w = Analytical_Gradien_Decent(train_x, train_y)
Test_Cost = 0.5 * np.sum(np.square(np.reshape(np.squeeze(np.matmul(test_x,w)) - test_y, (-1,1))))
print("Analytical Gradien_Decent W vector:")
print(w)
print('Analytical Test_Cost:', Test_Cost)

Analytical Gradien_Decent W vector:
[ 0.90056451  0.78629331  0.85104314  1.29889413  0.12989067  1.57224887
  0.99869359 -0.01519667]
Analytical Test_Cost: 23.36176447993874
